In [16]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
# print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
# print(f"DEST_DIR: {DEST_DIR}\n")

# # Build paths
# source_path = Path(DOWNLOAD_DIR) / f'df_OHLCV_{date_str}.parquet'
# dest_path = Path(DEST_DIR) / f'df_OHLCV_{date_str}_clean.parquet'

# print(f"source_path: {source_path}")
# print(f"dest_path: {dest_path}")

date_str: 2025-04-01


In [17]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

SOURCE_PATH_OHLCV = f'..\data\df_OHLCV_{date_str}_clean.parquet'
SOURCE_PATH = '..\data\df_finviz_n_ratios_merged.parquet'
PARQUET_PATH_COVARIANCE = '..\data\df_cov_emv_matrix.parquet'
PARQUET_PATH_CORRELATION = '..\data\df_corr_emv_matrix.parquet'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\setuptools\\_vendor', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


In [18]:
import pandas as pd

df_ohlcv = pd.read_parquet(SOURCE_PATH_OHLCV)
df_finviz = pd.read_parquet(SOURCE_PATH)

print(f"Loaded df_ohlcv shape: {df_ohlcv.shape}")
print(f"Loaded df_finviz shape: {df_finviz.shape}")


Loaded df_ohlcv shape: (386038, 9)
Loaded df_finviz shape: (1538, 54)


In [19]:
# Combine indices from both dataframes and remove duplicates
tickers = list(set(df_finviz.index))
print(f"Total unique tickers: {len(tickers)}")

Total unique tickers: 1538


In [20]:
df_close = df_ohlcv['Adj Close'].unstack(level=0)

# Assume `tickers` is your predefined list of tickers
common_tickers = df_close.columns.intersection(tickers)
df_close = df_close[common_tickers]

# Assuming `tickers` is your original list of tickers
missing_tickers = list(set(tickers) - set(df_close.columns))

# Print the missing tickers (if any)
if missing_tickers:
    print(f"{len(missing_tickers)} tickers not found in df_close:")
    print(missing_tickers)
else:
    print("All tickers in your list are present in df_close.")

print(f"\nShape of filtered df_close: {df_close.shape}")
display(df_close.head())

All tickers in your list are present in df_close.

Shape of filtered df_close: (251, 1538)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,...,YUMC,Z,ZBH,ZBRA,ZG,ZION,ZM,ZS,ZTO,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2024-04-02,143.41,35.24,14.40,84.95,168.05,174.34,2.3873,160.11,109.78,87.00,...,39.52,46.86,127.34,296.50,45.75,41.03,63.78,188.74,21.04,163.27
2024-04-03,142.97,36.76,14.16,85.97,168.85,171.10,2.4270,159.34,109.06,87.70,...,39.08,46.74,128.87,292.75,45.85,41.13,63.22,186.31,20.59,161.25
2024-04-04,140.59,35.55,13.83,85.33,168.03,162.00,2.4270,158.84,107.91,86.62,...,39.07,46.82,126.44,292.96,45.83,40.35,63.28,182.01,20.34,163.77
2024-04-05,143.36,36.08,13.76,89.58,168.78,164.02,2.3873,161.77,108.98,90.24,...,38.31,47.22,126.21,290.38,46.23,40.70,62.94,183.34,19.95,164.11
2024-04-08,143.70,35.93,13.90,92.53,167.66,163.83,2.3873,160.27,108.31,89.91,...,38.01,47.70,125.58,296.87,46.61,41.58,63.01,183.71,19.80,163.94


In [21]:
df_close_sorted = df_close.sort_index()
print(f'df_close_sorted shape: {df_close_sorted.shape}')
display(df_close_sorted.head())

# Calculate daily returns
df_close_returns = df_close_sorted.pct_change().dropna()
print(f'df_close_returns shape: {df_close_returns.shape}')
display(df_close_returns.head())


df_close_sorted shape: (251, 1538)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,...,YUMC,Z,ZBH,ZBRA,ZG,ZION,ZM,ZS,ZTO,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2024-04-02,143.41,35.24,14.40,84.95,168.05,174.34,2.3873,160.11,109.78,87.00,...,39.52,46.86,127.34,296.50,45.75,41.03,63.78,188.74,21.04,163.27
2024-04-03,142.97,36.76,14.16,85.97,168.85,171.10,2.4270,159.34,109.06,87.70,...,39.08,46.74,128.87,292.75,45.85,41.13,63.22,186.31,20.59,161.25
2024-04-04,140.59,35.55,13.83,85.33,168.03,162.00,2.4270,158.84,107.91,86.62,...,39.07,46.82,126.44,292.96,45.83,40.35,63.28,182.01,20.34,163.77
2024-04-05,143.36,36.08,13.76,89.58,168.78,164.02,2.3873,161.77,108.98,90.24,...,38.31,47.22,126.21,290.38,46.23,40.70,62.94,183.34,19.95,164.11
2024-04-08,143.70,35.93,13.90,92.53,167.66,163.83,2.3873,160.27,108.31,89.91,...,38.01,47.70,125.58,296.87,46.61,41.58,63.01,183.71,19.80,163.94


df_close_returns shape: (250, 1538)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,...,YUMC,Z,ZBH,ZBRA,ZG,ZION,ZM,ZS,ZTO,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2024-04-03,-0.003068,0.043133,-0.016667,0.012007,0.004760,-0.018584,0.016630,-0.004809,-0.006559,0.008046,...,-0.011134,-0.002561,0.012015,-0.012648,0.002186,0.002437,-0.008780,-0.012875,-0.021388,-0.012372
2024-04-04,-0.016647,-0.032916,-0.023305,-0.007444,-0.004856,-0.053185,0.000000,-0.003138,-0.010545,-0.012315,...,-0.000256,0.001712,-0.018856,0.000717,-0.000436,-0.018964,0.000949,-0.023080,-0.012142,0.015628
2024-04-05,0.019703,0.014909,-0.005061,0.049807,0.004463,0.012469,-0.016358,0.018446,0.009916,0.041792,...,-0.019452,0.008543,-0.001819,-0.008807,0.008728,0.008674,-0.005373,0.007307,-0.019174,0.002076
2024-04-08,0.002372,-0.004157,0.010174,0.032931,-0.006636,-0.001158,0.000000,-0.009272,-0.006148,-0.003657,...,-0.007831,0.010165,-0.004992,0.022350,0.008220,0.021622,0.001112,0.002018,-0.007519,-0.001036
2024-04-09,0.020320,0.013359,0.003597,-0.035556,0.007217,0.002014,0.016630,0.014912,0.017358,-0.035257,...,0.020258,0.006080,0.021341,0.031327,0.012444,0.012266,0.013331,0.026074,0.017677,0.008784


In [22]:
cov_emv_matrix, corr_emv_matrix = utils.get_cov_corr_ewm_matrices_chunked(df_close_returns, span=21, return_corr=True, return_cov=True)

print(f'cov_emv_matrix shape: {cov_emv_matrix.shape}')
display(cov_emv_matrix.head())
print(f'corr_emv_matrix shape: {corr_emv_matrix.shape}')
display(corr_emv_matrix.head())  

cov_emv_matrix shape: (1538, 1538)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,...,YUMC,Z,ZBH,ZBRA,ZG,ZION,ZM,ZS,ZTO,ZTS
Symbol,,,,,,,,,,,,,,,,,,,,,
A,0.000204,0.000148,0.000228,0.000267,0.000109,0.000046,0.000032,0.000092,0.000017,0.000063,...,0.000077,0.000122,0.000048,0.000148,0.000128,0.000132,0.000107,0.000105,0.000086,0.000064
AA,0.000148,0.000654,0.000242,0.000369,0.000120,-0.000068,0.000122,0.000235,-0.000136,0.000081,...,0.000155,0.000260,-0.000073,0.000158,0.000273,0.000242,0.000190,0.000304,0.000074,-0.000045
AAL,0.000228,0.000242,0.000726,0.000403,0.000203,0.000041,0.000049,0.000385,0.000088,0.000152,...,0.000164,0.000319,0.000088,0.000324,0.000330,0.000238,0.000212,0.000172,0.000183,0.000121
AAON,0.000267,0.000369,0.000403,0.001171,0.000199,-0.000102,-0.000044,0.000332,-0.000074,0.000039,...,0.000091,0.000455,-0.000024,0.000307,0.000466,0.000237,0.000261,0.000416,0.000087,-0.000088
AAPL,0.000109,0.000120,0.000203,0.000199,0.000267,-0.000020,0.000052,0.000196,0.000065,0.000080,...,0.000075,0.000188,0.000013,0.000128,0.000193,0.000148,0.000107,0.000135,0.000070,0.000066


corr_emv_matrix shape: (1538, 1538)


Symbol,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,...,YUMC,Z,ZBH,ZBRA,ZG,ZION,ZM,ZS,ZTO,ZTS
Symbol,,,,,,,,,,,,,,,,,,,,,
A,1.000000,0.406749,0.593011,0.545829,0.466261,0.232688,0.146637,0.271101,0.082090,0.344365,...,0.306069,0.401818,0.261713,0.617622,0.399777,0.587820,0.467140,0.307649,0.276840,0.333158
AA,0.406749,1.000000,0.351116,0.421992,0.287440,-0.193424,0.310617,0.384768,-0.369810,0.248200,...,0.343354,0.476127,-0.219861,0.367744,0.476878,0.600398,0.462815,0.498933,0.133321,-0.128516
AAL,0.593011,0.351116,1.000000,0.437124,0.461717,0.109400,0.117176,0.597962,0.225396,0.438346,...,0.343748,0.555112,0.252751,0.716622,0.547695,0.559250,0.489875,0.268333,0.313010,0.330970
AAON,0.545829,0.421992,0.437124,1.000000,0.355828,-0.215739,-0.084445,0.406231,-0.149337,0.089019,...,0.151097,0.622958,-0.055294,0.533739,0.607974,0.439540,0.475756,0.509967,0.116961,-0.188701
AAPL,0.466261,0.287440,0.461717,0.355828,1.000000,-0.088115,0.206969,0.502937,0.274775,0.383912,...,0.260024,0.540985,0.062364,0.465495,0.526679,0.572842,0.408184,0.347677,0.199139,0.299066


In [23]:
cov_emv_matrix.to_parquet(PARQUET_PATH_COVARIANCE)
print(f'cov_emv_matrix parqueted to {PARQUET_PATH_COVARIANCE}')
display(cov_emv_matrix.info())

corr_emv_matrix.to_parquet(PARQUET_PATH_CORRELATION)
print(f'corr_emv_matrix parqueted to {PARQUET_PATH_CORRELATION}')
display(corr_emv_matrix.info())

cov_emv_matrix parqueted to ..\data\df_cov_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1538 entries, A to ZTS
Columns: 1538 entries, A to ZTS
dtypes: float64(1538)
memory usage: 18.1+ MB


None

corr_emv_matrix parqueted to ..\data\df_corr_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1538 entries, A to ZTS
Columns: 1538 entries, A to ZTS
dtypes: float64(1538)
memory usage: 18.1+ MB


None